# Evaluating Tool Use with Limbic

This notebook evaluates how well Language Models (LLMs) use tools by testing them on the [Limbic Tool-Use Benchmark](https://huggingface.co/datasets/quotientai/mcp-tool-use-eval) dataset. We'll test Kimi K2 and Qwen models, analyze their performance, and visualize the results.

## Setup

You'll need API keys from [Together AI](https://www.together.ai) for model access and [Hugging Face](https://huggingface.co) for dataset access. Set them in the environment variables below.

In [ ]:
os.environ['TOGETHER_API_KEY'] = "your_together_api_key"
os.environ['HUGGINGFACE_TOKEN'] = "your_huggingface_token"

### 2. Install Required Packages

Next, install the necessary Python packages:

In [ ]:
%pip install together datasets requests pandas numpy matplotlib seaborn tqdm

### 3. Import Dependencies

We'll use these libraries throughout the notebook:

In [ ]:
import json
import time
from typing import Dict, List, Any, Optional
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from collections import Counter
from tqdm import tqdm

from datasets import load_dataset
from together import Together

## Dataset

The [Limbic Tool-Use Benchmark](https://huggingface.co/datasets/quotientai/mcp-tool-use-eval) contains 1,000 test cases that evaluate how well models handle tool selection, parameter usage, and value validation. Each case includes a user request, system instructions, and available tools for the model to use.

In [5]:
# Load dataset
data = load_dataset("quotientai/mcp-tool-use-eval")

# Check available splits
print(data)

# Inspect the first entry
first = data["test"][0]
print("\nFirst entry structure:")
print("Keys:", first.keys())

print("\nUser Prompt:", first["user_prompt"])
print("\nSystem Prompt:", first["system_prompt"])

# Parse and inspect available tools
tools = json.loads(first["available_tools"])
print("\nAvailable Tools:")
for t in tools[:2]:  # show only first 2 tools for readability
    print(json.dumps(t, indent=2))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/426 [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['user_prompt', 'system_prompt', 'available_tools'],
        num_rows: 1000
    })
})

First entry structure:
Keys: dict_keys(['user_prompt', 'system_prompt', 'available_tools'])

User Prompt: I'm conducting a thorough analysis of the latest popular apps in the health and fitness category. Can you provide me with the privacy details for the app with ID 444934666? I want to ensure that it aligns with our privacy standards before recommending it to our users.

System Prompt: Ensure that the requested app ID is valid and that the privacy details include information on data collection and sharing practices.

Available Tools:
{
  "name": "google-play-developer",
  "description": "Get apps by a developer on Google Play",
  "inputSchema": {
    "type": "object",
    "$schema": "http://json-schema.org/draft-07/schema#",
    "required": [
      "devId"
    ],
    "properties": {
      "num": {
        "type": "number",
        "default": 60,
 

## Helper Functions

This section defines three core functions for the evaluation pipeline: one to format tools for the Together AI API, another to call the models with the formatted tools, and a third to evaluate the model responses using Limbic's API. Each function includes error handling for API limits, timeouts, and malformed responses.

In [6]:
def format_tool(tool):
    """Format tool to match Together's expected schema"""
    schema = tool.get("inputSchema", tool.get("parameters", {}))

    return {
        "type": "function",
        "function": {
            "name": tool.get("name"),
            "description": tool.get("description", ""),
            "parameters": schema  # This becomes 'parameters' for the API
        }
    }

def call_together_api(
    user_prompt: str,
    model: str,
    system_prompt: Optional[str] = None,
    tools: Optional[list] = None,
    temperature: float = 0.6
) -> Dict[str, Any]:
    """Call Together API with support for different models and tools"""
    client = Together()

    # Build messages
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    # Build API parameters
    params = {
        "model": model,
        "messages": messages,
        "temperature": temperature
    }

    # Add tools if provided
    if tools:
        formatted_tools = [format_tool(tool) for tool in tools]
        params["tools"] = formatted_tools
        params["tool_choice"] = "auto"

    try:
        # Make API call
        response = client.chat.completions.create(**params)

        # Handle regular response
        tool_calls = []
        if hasattr(response.choices[0].message, 'tool_calls') and response.choices[0].message.tool_calls:
            for tool_call in response.choices[0].message.tool_calls:
                tool_calls.append({
                    "type": "tool_use",
                    "id": tool_call.id,
                    "name": tool_call.function.name,
                    "input": json.loads(tool_call.function.arguments)
                })

        return {
            "success": True,
            "messages": [
                {"role": "user", "content": user_prompt},
                {"role": "assistant", "content": tool_calls if tool_calls else None}
            ]
        }
    except Exception as e:
        print(f"Error calling model: {str(e)[:100]}")
        return {
            "success": False,
            "error": str(e)
        }

## Evaluation

Limbic evaluates each model response and assigns one of four labels: correct, incorrect_tool, incorrect_parameter_names, or incorrect_parameter_values. This provides detailed feedback about where models succeed or fail in their tool use.

In [7]:
def get_tool_call_eval(messages: List[dict], available_tools: List[dict]) -> Dict[str, Any]:
    """Limbic API evaluation function"""
    payload = {
        "messages": messages,
        "available_tools": available_tools
    }

    url = "https://quotient-ai--tool-call-evaluator-7b-api-v0-fastapi-app.modal.run/api/v1/detections/tool-use"
    try:
        response = requests.post(url, json=payload, timeout=120)
        if response.status_code == 200:
            result = response.json()
            return {
                "success": True,
                "status_code": response.status_code,
                "score": result["score"],
                "reasoning": result["reasoning"],
            }
        else:
            return {
                "success": False,
                "status_code": response.status_code,
                "error": response.text,
            }
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
        }


## Model Evaluation

We test two models from Together AI: Kimi K2 (instruction-following model) and Qwen (multilingual model optimized for structured outputs). For each test case, we prepare the data, call the model, evaluate its response with Limbic, and track metrics. Rate limiting and error handling ensure reliable results.

In [8]:
# Initialize models configuration
models = [
    {
        "name": "Kimi K2",
        "model": "moonshotai/Kimi-K2-Instruct"
    },
    {
        "name": "Qwen",
        "model": "Qwen/Qwen3-235B-A22B-Instruct-2507-tput"
    }
]

# Initialize score tracking
score_categories = [
    "correct",
    "incorrect_tool",
    "incorrect_parameter_names",
    "incorrect_parameter_values"
]

scores = {
    model["name"]: {
        category: 0 for category in score_categories
    } for model in models
}

# Store detailed results
detailed_results = []

random_subset = data["test"].shuffle(seed=42).select(range(len(data["test"])))
total_examples = len(random_subset)

# Process examples and evaluate with both models
print(f"Processing {total_examples} random examples...")
print("-" * 50)

for i, entry in enumerate(tqdm(random_subset, desc="Evaluating examples"), 1):
    user_prompt = entry["user_prompt"]
    test_type = entry.get("mixture_type", "unknown")
    tools = json.loads(entry["available_tools"])  # Parse JSON string

    # Only print first 5
    show = i <= 5
    if show:
        print(f"\nExample {i}:")
        print(f"User Prompt: {user_prompt[:100]}...")

    for model_config in models:
        if show:
            print(f"\n{model_config['name']} Response:")

        try:
            # Call model
            result = call_together_api(
                user_prompt=user_prompt,
                system_prompt=entry.get("system_prompt"),  # use if present
                model=model_config["model"],
                tools=tools
            )

            tool_calls = result["messages"][1]["content"]

            if show:
                if tool_calls is None:
                    print("null")
                else:
                    print(json.dumps(tool_calls, indent=2))

            # Evaluate with Limbic
            limbic_result = get_tool_call_eval(result["messages"], tools)

            if show:
                print("\nLimbic Evaluation:")
                if limbic_result["success"]:
                    print(f"Score: {limbic_result['score']}")
                    print(f"Reasoning: {limbic_result['reasoning'][:200]}...")
                else:
                    print(f"Error: {limbic_result.get('error', 'Unknown error')}")

            # Track metrics
            if limbic_result["success"]:
                score = limbic_result["score"]
                if score in scores[model_config["name"]]:
                    scores[model_config["name"]][score] += 1

                detailed_results.append({
                    'example_id': i,
                    'test_type': test_type,
                    'model': model_config["name"],
                    'user_prompt': user_prompt,
                    'tool_called': tool_calls[0]["name"] if tool_calls else "NO_TOOL_CALLED",
                    'tool_params': tool_calls[0]["input"] if tool_calls else None,
                    'limbic_score': score,
                    'limbic_reasoning': limbic_result.get("reasoning", []),
                    'was_null_response': tool_calls is None,
                    'ground_truth': entry.get("ground_truth_tool_call")
                })

        except Exception as e:
            if show:
                print(f"Error: {str(e)}")

    time.sleep(0.5)  # Rate limiting

Processing 1000 random examples...
--------------------------------------------------


Evaluating examples:   0%|          | 0/1000 [00:00<?, ?it/s]


Example 1:
User Prompt: I need to batch create several new contacts for my CRM. I'm working on importing data from an old sy...

Kimi K2 Response:
null

Limbic Evaluation:
Score: incorrect_tool
Reasoning: ['no tool selected, correct tool was available']...

Qwen Response:
[
  {
    "type": "tool_use",
    "id": "call_b6ehu10auxr88h89387dtk29",
    "name": "crm_batch_create_objects",
    "input": {
      "inputs": [
        {
          "associations": [
            {
              "to": {
                "id": "12345"
              },
              "types": [
                {
                  "associationCategory": "HUBSPOT_DEFINED",
                  "associationTypeId": 1
                }
              ]
            }
          ],
          "properties": {
            "email": "john.doe@example.com",
            "firstname": "John",
            "lastname": "Doe"
          }
        },
        {
          "associations": [
            {
              "to": {
                "id": "

Evaluating examples:   0%|          | 1/1000 [00:10<2:56:18, 10.59s/it]


Example 2:
User Prompt: I noticed that our 'web-server-01' host has been muted due to a recent incident. Can you please unmu...

Kimi K2 Response:
null

Limbic Evaluation:
Score: correct
Reasoning: []...

Qwen Response:
null

Limbic Evaluation:
Score: correct
Reasoning: []...


...

Evaluating examples:  80%|████████  | 801/1000 [2:43:28<38:17, 11.55s/it]

Error calling model: Error code: 429 - {"message": "You are on tier Build Tier 1, which offers 6 queries and 180000 token


Evaluating examples:  80%|████████  | 802/1000 [2:43:53<51:06, 15.49s/it]

Error calling model: Error code: 429 - {"message": "You are on tier Build Tier 1, which offers 6 queries and 180000 token


Evaluating examples: 100%|██████████| 1000/1000 [3:24:13<00:00, 12.25s/it]


In [11]:
len(detailed_results)

1997

In [12]:
# save_results_jsonl.py
import json
from pathlib import Path
from typing import List, Dict, Any

def save_jsonl(detailed_results: List[Dict[str, Any]], out_path: str) -> str:
    """
    Save detailed evaluation results to a JSON Lines file.
    Each element of `detailed_results` is written as a single line (compact JSON).
    """
    path = Path(out_path)
    path.parent.mkdir(parents=True, exist_ok=True)
    with path.open("w", encoding="utf-8") as f:
        for row in detailed_results:
            f.write(json.dumps(row, ensure_ascii=False) + "\n")
    return str(path)

out_file = save_jsonl(detailed_results, "out/evaluation_results.jsonl")
print(f"Saved: {out_file}")


Saved: out/evaluation_results.jsonl


## Results Aggregation

We aggregate Limbic’s outputs into error distributions by model and test type.
The visualization highlights where models succeed, and which error categories dominate.

In [9]:
# Print aggregated results
print("\nAggregated Results:")
print("-" * 50)
print(f"Total examples processed: {total_examples}\n")

for model_name, model_scores in scores.items():
    print(f"\n{model_name} Results:")
    print("-" * 20)
    for category, count in model_scores.items():
        percentage = (count / total_examples) * 100
        print(f"{category}: {count} ({percentage:.1f}%)")

# Create a summary
print("\nScore Distribution:")
print("-" * 50)
for model_name, model_scores in scores.items():
    print(f"\n{model_name}:")
    max_bar_length = 40
    for category, count in model_scores.items():
        percentage = (count / total_examples) * 100
        bar_length = int((count / total_examples) * max_bar_length)
        bar = "█" * bar_length
        print(f"{category:25} [{bar:<40}] {percentage:.1f}%")


Aggregated Results:
--------------------------------------------------
Total examples processed: 1000


Kimi K2 Results:
--------------------
correct: 813 (81.3%)
incorrect_tool: 146 (14.6%)
incorrect_parameter_names: 23 (2.3%)
incorrect_parameter_values: 16 (1.6%)

Qwen Results:
--------------------
correct: 848 (84.8%)
incorrect_tool: 115 (11.5%)
incorrect_parameter_names: 23 (2.3%)
incorrect_parameter_values: 13 (1.3%)

Score Distribution:
--------------------------------------------------

Kimi K2:
correct                   [████████████████████████████████        ] 81.3%
incorrect_tool            [█████                                   ] 14.6%
incorrect_parameter_names [                                        ] 2.3%
incorrect_parameter_values [                                        ] 1.6%

Qwen:
correct                   [█████████████████████████████████       ] 84.8%
incorrect_tool            [████                                    ] 11.5%
incorrect_parameter_names [        

## Analysis

This section calculates key metrics and analyzes error patterns to understand model performance. We look at overall accuracy, error distributions, and specific failure cases to identify areas for improvement.

In [15]:
# Calculate key metrics
total_evaluations = len(results_df)
unique_examples = results_df['example_id'].nunique()

print(f"\nDataset Overview:")
print(f"- Examples tested: {unique_examples}")
print(f"- Models evaluated: {results_df['model'].nunique()}")
print(f"- Total evaluations: {total_evaluations}")

print(f"\nPerformance Summary:")
for model in results_df['model'].unique():
    model_df = results_df[results_df['model'] == model]
    accuracy = (model_df['limbic_score'] == 'correct').mean() * 100
    null_rate = model_df['was_null_response'].mean() * 100
    print(f"\n{model}:")
    print(f"  - Accuracy: {accuracy:.1f}%")
    print(f"  - Null response rate: {null_rate:.1f}%")

    # Most common error
    errors = model_df[model_df['limbic_score'] != 'correct']['limbic_score']
    if len(errors) > 0:
        print(f"  - Most common error: {errors.mode()[0]}")

print(f"\nKey Findings:")
# Null response insight
null_responses = results_df[results_df['was_null_response'] == True]
if len(null_responses) > 0:
    print(f"- {len(null_responses)} cases where models didn't call any tool")
    null_correct = null_responses[null_responses['limbic_score'] == 'correct']
    print(f"  - {len(null_correct)} were correctly identified as not needing tools")
    null_wrong = null_responses[null_responses['limbic_score'] != 'correct']
    print(f"  - {len(null_wrong)} should have called a tool but didn't")

# Error patterns
print(f"\nError Patterns:")
error_dist = results_df[results_df['limbic_score'] != 'correct']['limbic_score'].value_counts()
for error_type, count in error_dist.items():
    examples = results_df[results_df['limbic_score'] == error_type]['test_type'].value_counts().head(1)
    if len(examples) > 0:
        print(f"- {error_type}: {count} occurrences, most common in '{examples.index[0]}' tests")


Dataset Overview:
- Examples tested: 1000
- Models evaluated: 2
- Total evaluations: 1997

Performance Summary:

Kimi K2:
  - Accuracy: 81.5%
  - Null response rate: 44.6%
  - Most common error: incorrect_tool

Qwen:
  - Accuracy: 84.9%
  - Null response rate: 35.1%
  - Most common error: incorrect_tool

Key Findings:
- 796 cases where models didn't call any tool
  - 689 were correctly identified as not needing tools
  - 107 should have called a tool but didn't

Error Patterns:
- incorrect_tool: 261 occurrences, most common in 'unknown' tests
- incorrect_parameter_names: 46 occurrences, most common in 'unknown' tests
- incorrect_parameter_values: 29 occurrences, most common in 'unknown' tests


## Failure Analysis

This section examines specific failure patterns by comparing model responses. We analyze cases where one model succeeds while the other fails, instances where both models fail differently, and situations where models choose not to use tools. This helps identify systematic issues and opportunities for improvement in both model training and tool design.

### Key Findings
- Qwen shows better accuracy (84.9% vs 81.5%) with fewer tool selection errors
- Tool selection is the main challenge (11-15% of errors)
- Parameter errors are rare (2-3% naming, 1-2% values)
- Null responses vary between models and are mostly correct

### Areas for Improvement
- **Model Development:** Focus on tool selection accuracy and parameter validation
- **Tool Design:** Clearer descriptions and consistent parameter naming
- **Evaluation:** Add more edge cases and multi-tool scenarios

In [22]:
import pandas as pd
import json

def normalize_reasoning(x):
    """Ensure limbic_reasoning is always a list of strings."""
    if isinstance(x, list):
        return [str(i) for i in x]
    if isinstance(x, str):
        try:
            parsed = json.loads(x)
            if isinstance(parsed, list):
                return [str(i) for i in parsed]
            return [x]
        except Exception:
            return [x]
    return []

def analyze_failure_patterns(results_df, models=("Kimi K2", "Qwen")):
    # Normalize schema
    results_df = results_df.copy()
    if "tool_called" not in results_df.columns:
        results_df["tool_called"] = "N/A"
    results_df["was_null_response"] = results_df["was_null_response"].astype(bool)
    results_df["limbic_reasoning"] = results_df["limbic_reasoning"].apply(normalize_reasoning)

    # Ensure one row per (example_id, model)
    results_df = (
        results_df.sort_values(by=["example_id", "model"])
        .drop_duplicates(subset=["example_id", "model"], keep="last")
    )

    # 1. Failures in first model but success in second
    print("-"*50)
    print(f"FAILURES IN {models[0]} BUT SUCCESS IN {models[1]}:")
    print("-"*50)
    for ex_id, grp in results_df.groupby("example_id"):
        if set(models).issubset(grp["model"].unique()):
            row_a = grp[grp["model"] == models[0]].iloc[0]
            row_b = grp[grp["model"] == models[1]].iloc[0]
            if row_a["limbic_score"] != "correct" and row_b["limbic_score"] == "correct":
                print(f"\nExample {ex_id}:")
                print(f"Prompt: {row_a['user_prompt'][:100]}...")
                print(f"{models[0]} failed: {row_a['limbic_score']} - {', '.join(row_a['limbic_reasoning'])}")
                print(f"{models[1]} succeeded: tool={row_b['tool_called']}")

    # 2. Failures in both
    print("\n" + "-"*50)
    print("FAILURES IN BOTH MODELS:")
    print("-"*50)
    for ex_id, grp in results_df.groupby("example_id"):
        if set(models).issubset(grp["model"].unique()):
            row_a = grp[grp["model"] == models[0]].iloc[0]
            row_b = grp[grp["model"] == models[1]].iloc[0]
            if row_a["limbic_score"] != "correct" and row_b["limbic_score"] != "correct":
                print(f"\nExample {ex_id}:")
                print(f"Prompt: {row_a['user_prompt'][:100]}...")
                print(f"{models[0]}: {row_a['limbic_score']} - {', '.join(row_a['limbic_reasoning'])}")
                print(f"{models[1]}: {row_b['limbic_score']} - {', '.join(row_b['limbic_reasoning'])}")

    # 3. Null responses
    print("\n" + "-"*50)
    print("NULL RESPONSES:")
    print("-"*50)
    nulls = results_df[results_df["was_null_response"]]
    if not nulls.empty:
        for _, row in nulls.iterrows():
            print(f"\nModel: {row['model']}, Example {row['example_id']}")
            print(f"Prompt: {row['user_prompt'][:100]}...")
            print(f"Score: {row['limbic_score']} | Reasoning: {', '.join(row['limbic_reasoning'])}")
    else:
        print("None found.")


In [23]:
results_df = pd.DataFrame(detailed_results)
analyze_failure_patterns(results_df)

--------------------------------------------------
FAILURES IN Kimi K2 BUT SUCCESS IN Qwen:
--------------------------------------------------

Example 1:
Prompt: I need to batch create several new contacts for my CRM. I'm working on importing data from an old sy...
Kimi K2 failed: incorrect_tool - no tool selected, correct tool was available
Qwen succeeded: tool=crm_batch_create_objects

Example 57:
Prompt: I have just completed a new programming module that automates data entry tasks. Before deploying it,...
Kimi K2 failed: incorrect_tool - no tool selected, correct tool was available
Qwen succeeded: tool=NO_TOOL_CALLED

Example 60:
Prompt: I need to fetch all the canned responses from the 'Customer Feedback' folder in Freshdesk to help ou...
Kimi K2 failed: incorrect_tool - no tool selected, correct tool was available
Qwen succeeded: tool=list_canned_responses

Example 69:
Prompt: Hey team, can we update the name of the 'Games' category to 'Updated Games Category'? Also, let's mo...

In [20]:
print("\n" + "-"*50)
print("ERROR DISTRIBUTION BY MODEL:")
print("-"*50)
error_summary = pd.crosstab(
    results_df[results_df["limbic_score"] != "correct"]["limbic_score"],
    results_df[results_df["limbic_score"] != "correct"]["model"]
)
print(error_summary if not error_summary.empty else "No errors")

results_df


--------------------------------------------------
ERROR DISTRIBUTION BY MODEL:
--------------------------------------------------
model                       Kimi K2  Qwen
limbic_score                             
incorrect_parameter_names        23    23
incorrect_parameter_values       16    13
incorrect_tool                  146   115


,example_id,test_type,model,user_prompt,tool_called,tool_params,limbic_score,limbic_reasoning,was_null_response,ground_truth
0,1,unknown,Kimi K2,I need to batch create several new contacts fo...,NO_TOOL_CALLED,None,incorrect_tool,"[no tool selected, correct tool was available]",True,None
1,1,unknown,Qwen,I need to batch create several new contacts fo...,crm_batch_create_objects,{'inputs': [{'associations': [{'to': {'id': '1...,correct,[],False,None
2,2,unknown,Kimi K2,I noticed that our 'web-server-01' host has be...,NO_TOOL_CALLED,None,correct,[],True,None
3,2,unknown,Qwen,I noticed that our 'web-server-01' host has be...,NO_TOOL_CALLED,None,correct,[],True,None
4,3,unknown,Kimi K2,Can you show me the contents of the images dir...,NO_TOOL_CALLED,None,correct,[],True,None
...,...,...,...,...,...,...,...,...,...,...
1992,998,unknown,Qwen,I'm looking to connect with software engineers...,openai_computer_use_agent,"{'maxSteps': 25, 'task': 'Search for software ...",correct,[],False,None
1993,999,unknown,Kimi K2,I'm working on a data analysis project and I n...,NO_TOOL_CALLED,None,correct,[],True,None
1994,999,unknown,Qwen,I'm working on a data analysis project and I n...,geminithinking,"{'nextThoughtNeeded': True, 'query': 'How to l...",incorrect_tool,[incorrect function selected from available op...,False,None
1995,1000,unknown,Kimi K2,I'm managing my team's project on Monday.com a...,monday-list-subitems-in-items,"{'itemIds': ['123456', '789101', '112131']}",correct,[],False,None
